In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [7]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [8]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [9]:
pd.read_sql(
    '''
    SELECT
        'CRIM' as column, SUM(CRIM IS NULL) as null_count
    FROM boston
    UNION ALL
    SELECT
        'ZN', SUM(ZN IS NULL)
    FROM boston
    UNION ALL
    SELECT
        'INDUS', SUM(INDUS IS NULL)
    FROM boston
    UNION ALL
    SELECT
        'CHAS', SUM(CHAS IS NULL)
    FROM boston
    UNION ALL
    SELECT
        'NOX', SUM(NOX IS NULL)
    FROM boston
    ''',
    con
)


,column,null_count
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [10]:
pd.read_sql(
    '''
    SELECT
        'CRIM' as column, COUNT(DISTINCT CRIM) as unique_count
    FROM boston
    UNION ALL
    SELECT
        'ZN', COUNT(DISTINCT ZN)
    FROM boston
    UNION ALL
    SELECT
        'INDUS', COUNT(DISTINCT INDUS)
    FROM boston
    UNION ALL
    SELECT
        'CHAS', COUNT(DISTINCT CHAS)
    FROM boston
    UNION ALL
    SELECT
        'NOX', COUNT(DISTINCT NOX)
    FROM boston
    ''',
    con
)


,column,unique_count
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [23]:
pd.read_sql(
    '''
    SELECT column_name
    FROM (
        SELECT 'CRIM' as column_name, MIN(CRIM) as min_val, 
               (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) as median_val
        FROM boston
        UNION ALL
        SELECT 'ZN', MIN(ZN), 
               (SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2)
        FROM boston
        UNION ALL
        SELECT 'INDUS', MIN(INDUS), 
               (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2)
        FROM boston
        UNION ALL
        SELECT 'CHAS', MIN(CHAS), 
               (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2)
        FROM boston
        UNION ALL
        SELECT 'NOX', MIN(NOX), 
               (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2)
        FROM boston
    )
    WHERE min_val = median_val
    ''',
    con
)


,column_name
0,ZN
1,CHAS


4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [12]:
pd.read_sql(
    '''
    WITH ranks AS (
        SELECT 
            LSTAT, 
            MEDV,
            RANK() OVER (ORDER BY LSTAT) as rank
        FROM boston
    )
    SELECT LSTAT, MEDV, rank
    FROM ranks
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    ''',
    con
)


,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [33]:
pd.read_sql(
    '''
    SELECT 
        CHAS as "Граничит с рекой", 
        AVG(MEDV) as "Средняя стоимость"
    FROM boston
    GROUP BY CHAS
    ''',
    con
)


,Граничит с рекой,Средняя стоимость
0,0.0,22.093843
1,1.0,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [19]:
pd.read_sql(
    '''
    WITH MedianValues AS (
    SELECT 
        CRIM, ZN, INDUS, CHAS, NOX,
        ROW_NUMBER() OVER (ORDER BY CRIM) as crim_row,
        ROW_NUMBER() OVER (ORDER BY ZN) as zn_row,
        ROW_NUMBER() OVER (ORDER BY INDUS) as indus_row,
        ROW_NUMBER() OVER (ORDER BY CHAS) as chas_row,
        ROW_NUMBER() OVER (ORDER BY NOX) as nox_row,
        (SELECT COUNT(*) FROM boston) as row_count
    FROM boston
),
MedianComparison AS (
    SELECT
        'CRIM' as column_name, 
        (SELECT MIN(CRIM) FROM boston) as min_val, 
        (SELECT CRIM FROM MedianValues WHERE crim_row = (row_count + 1) / 2) as median_val
    UNION ALL
    SELECT 
        'ZN', 
        (SELECT MIN(ZN) FROM boston), 
        (SELECT ZN FROM MedianValues WHERE zn_row = (row_count + 1) / 2)
    UNION ALL
    SELECT 
        'INDUS', 
        (SELECT MIN(INDUS) FROM boston), 
        (SELECT INDUS FROM MedianValues WHERE indus_row = (row_count + 1) / 2)
    UNION ALL
    SELECT 
        'CHAS', 
        (SELECT MIN(CHAS) FROM boston), 
        (SELECT CHAS FROM MedianValues WHERE chas_row = (row_count + 1) / 2)
    UNION ALL
    SELECT 
        'NOX', 
        (SELECT MIN(NOX) FROM boston), 
        (SELECT NOX FROM MedianValues WHERE nox_row = (row_count + 1) / 2)
)
SELECT column_name
FROM MedianComparison
WHERE min_val = median_val;

    ''',
    con
)


,column_name
0,ZN
1,CHAS


7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [34]:
import numpy as np

indus_percentiles = np.percentile(data['INDUS'].dropna(), [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
nox_percentiles = np.percentile(data['NOX'].dropna(), [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

indus_percentiles, nox_percentiles


(array([ 2.91,  4.39,  5.96,  7.38,  9.69, 12.83, 18.1 , 18.1 , 19.58,
        27.74]),
 array([0.427, 0.442, 0.472, 0.507, 0.538, 0.575, 0.605, 0.668, 0.713,
        0.871]))